In [43]:
import numpy as np
from urllib.request import urlopen
import scipy.optimize
import random
from math import exp
from math import log

In [45]:
def parseData(fname):
  for l in urlopen(fname):
    yield eval(l)

print("Reading data...")
data = list(parseData("http://jmcauley.ucsd.edu/cse190/data/beer/beer_50000.json"))
print("done")

def feature(datum):
  feat = [1, datum['review/taste'], datum['review/appearance'], datum['review/aroma'], datum['review/palate'], datum['review/overall']]
  return feat

X = [feature(d) for d in data]
y = [d['beer/ABV'] >= 6.5 for d in data]

def inner(x,y):
  return sum([x[i]*y[i] for i in range(len(x))])

def sigmoid(x):
  return 1.0 / (1 + exp(-x))

Reading data...
done


In [47]:
##################################################
# Logistic regression by gradient ascent         #
##################################################

# NEGATIVE Log-likelihood
def f(theta, X, y, lam):
  loglikelihood = 0
  for i in range(len(X)):
    logit = inner(X[i], theta)
    loglikelihood -= log(1 + exp(-logit))
    if not y[i]:
      loglikelihood -= logit
  for k in range(len(theta)):
    loglikelihood -= lam * theta[k]*theta[k]
  # for debugging
  # print("ll =" + str(loglikelihood))
  return -loglikelihood

# NEGATIVE Derivative of log-likelihood
def fprime(theta, X, y, lam):
  dl = [0]*len(theta)
  for i in range(len(X)):
    logit = inner(X[i], theta)
    for k in range(len(theta)):
      dl[k] += X[i][k] * (1 - sigmoid(logit))
      if not y[i]:
        dl[k] -= X[i][k]
  for k in range(len(theta)):
    dl[k] -= lam*2*theta[k]
  return numpy.array([-x for x in dl])


In [51]:
X_train = X
y_train = y

In [83]:

# split into 1/3 train, 1/3 validation, 1/3 test

Z = list(zip(X, y))

random.shuffle(Z)

x_shuffled, y_shuffled = zip(*Z)

print("X shuffled: ", np.shape(x_shuffled))
print("y shuffled: ", np.shape(y_shuffled))

samples = len(x_shuffled)

X_train = x_shuffled[0:round(samples/3)];
y_train = y_shuffled[0:round(samples/3)];

X_validation = x_shuffled[round(samples/3) + 1: 2 * round(samples/3)]
y_validation = y_shuffled[round(samples/3) + 1: 2 * round(samples/3)]

X_test = x_shuffled[2 * round(samples/3) + 1:samples]
y_test = y_shuffled[2 * round(samples/3) + 1:samples]

print("x train: ", np.shape(X_train), "x validate: ", np.shape(X_validate), "x test: ", np.shape(X_test))
print("y train: ", np.shape(y_train), "y validate: ", np.shape(y_validate), "y test: ", np.shape(y_test))



X shuffled:  (50000, 6)
y shuffled:  (50000,)
x train:  (16667, 6) x validate:  (16666, 6) x test:  (16665, 6)
y train:  (16667,) y validate:  (16666,) y test:  (16665,)


In [49]:
##################################################
# Train                                          #
##################################################

def train(lam):
  theta,_,_ = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X[0]), fprime, pgtol = 10, args = (X_train, y_train, lam))
  return theta


In [77]:
##################################################
# Predict                                        #
##################################################

def performance(theta, X, y):
  scores = [inner(theta,x) for x in X]
  predictions = [s > 0 for s in scores]
    
  print("Samples: ", len(X), "\n");
  print("Positives: ", sum(predictions))
  print("Negatives: ", len(predictions) - sum(predictions), "\n")
    
  correct = [(a==b) for (a,b) in zip(predictions, y)]
    
  truePositive = sum(correct)
  trueNegative = len(correct) - sum(correct)
  print("True Positives: ", truePositive)
  print("True Positives: ", trueNegative, "\n")
    
  falsePositive = sum([(a==1 and b==0) for (a,b) in zip(predictions,y_train)])
  falseNegative = sum([(a==0 and b==1) for (a,b) in zip(predictions,y_train)])
 
  print("False Positives: ", falsePositive)
  print("False Negatives: ", falseNegative, "\n")
 
  acc = sum(correct) * 1.0 / len(correct)
  return acc


In [72]:
##################################################
# Validation pipeline                            #
##################################################
lam = 1.0
theta = train(lam)

In [92]:
labels  = ["Training Set", "Validation Set", "Testing Set"]
corpusX = [X_train, X_validation, X_test]
corpusY = [y_train, y_validation, y_test]

for (label, x, y) in zip(labels, corpusX, corpusY):
    print("==================================================")
    print(label, "\n")
    acc = performance(theta, x, y)
    print("lambda = " + str(lam) + ":\taccuracy=" + str(acc), "\n")


Training Set 

Samples:  16667 

Positives:  12407
Negatives:  4260 

True Positives:  11924
True Positives:  4743 

False Positives:  3358
False Negatives:  1385 

lambda = 1.0:	accuracy=0.7154256914861703 

Validation Set 

Samples:  16666 

Positives:  12336
Negatives:  4330 

True Positives:  12009
True Positives:  4657 

False Positives:  4679
False Negatives:  2776 

lambda = 1.0:	accuracy=0.7205688227529101 

Testing Set 

Samples:  16665 

Positives:  12225
Negatives:  4440 

True Positives:  12014
True Positives:  4651 

False Positives:  4593
False Negatives:  2800 

lambda = 1.0:	accuracy=0.7209120912091209 



In [93]:
lambdas = [0, 0.01, 0.1, 1, 100]
labels  = ["Training Set", "Validation Set", "Testing Set"]
corpusX = [X_train, X_validation, X_test]
corpusY = [y_train, y_validation, y_test]

for lam in lambdas:
    theta = train(lam)
    for (x, y) in zip(corpusX, corpusY):
        print("==================================================")
        print(label, "\n")
        acc = performance(theta, x, y)
        print("lambda = " + str(lam) + ":\taccuracy=" + str(acc), "\n")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
    
        

Testing Set 

Samples:  16667 

Positives:  12481
Negatives:  4186 

True Positives:  11922
True Positives:  4745 

False Positives:  3396
False Negatives:  1349 

lambda = 0:	accuracy=0.7153056938861223 

Testing Set 

Samples:  16666 

Positives:  12423
Negatives:  4243 

True Positives:  11976
True Positives:  4690 

False Positives:  4710
False Negatives:  2720 

lambda = 0:	accuracy=0.718588743549742 

Testing Set 

Samples:  16665 

Positives:  12321
Negatives:  4344 

True Positives:  11956
True Positives:  4709 

False Positives:  4641
False Negatives:  2752 

lambda = 0:	accuracy=0.7174317431743175 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Testing Set 

Samples:  16667 

Positives:  12482
Negatives:  4185 

True Positives:  11921
True Positives:  4746 

False Positives:  3397
False Negatives:  1349 

lambda = 0.01:	accuracy=0.7152456950860983 

Testing Set 

Samples:  16666 

Positives:  12422
Negatives:  4244 

Tru